# Poisson Equation

Poisson Equation is defined as:

$$-\nabla^2U=1$$

In [2]:
# import necessary python modules
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

g:\software\Anaconda\envs\yolov5\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# define nn
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()

        self.fc1 = nn.Linear(1, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, 1)

    def forward(self, inputs):
        x = torch.tanh(self.fc1(inputs))
        x = torch.tanh(self.fc2(x))
        x = self.fc3(x)
        return x

In [8]:
# define loss function
def poisson_loss(u_pred, x_pred):
    u_pred = u_pred.sum()
    u_x = torch.autograd.grad(u_pred, x_pred, create_graph=True)[0]
    u_x = u_x.sum()
    u_xx = torch.autograd.grad(u_x, x_pred, create_graph=True)[0]
    f_pred = u_xx
    return torch.mean(torch.square(f_pred))

In [5]:
# define boundary condition
def boundary_condition(x):
    return np.sin(np.pi * x)

In [6]:
# start training
def train_PINN(x_data, u_data, num_epochs, device):

    model = PINN().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    x_data_tensor = torch.tensor(x_data, dtype=torch.float32, device=device, requires_grad=True)
    u_data_tensor = torch.tensor(u_data, dtype=torch.float32, device=device)

    for epoch in range(num_epochs):
        optimizer.zero_grad()

        u_pred = model(x_data_tensor)
        loss = poisson_loss(u_pred, x_data_tensor)

        loss.backward()
        optimizer.step()

        if epoch % 100 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')

    return model

In [ ]:
# train data
x_train = np.linspace(0, 1, 100).reshape(-1, 1) # (100,) -> (100, 1)
u_train = boundary_condition(x_train)

# append boundary condition
x_train = np.vstack((x_train, [[0], [1]])) # (100, 1) -> (102, 1)
u_train = np.vstack((u_train, [[boundary_condition(0)], [boundary_condition(1)]]))

num_epochs = 1000
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
trained_model = train_PINN(x_train, u_train, num_epochs, device)

In [ ]:
import torch

x = torch.tensor([5.0, 7.0], requires_grad=True)
y = x**2

loss1 = torch.mean(y)

h1 = torch.autograd.grad(y[0], x, retain_graph = True, create_graph=True)
h2 = torch.autograd.grad(y[1], x, retain_graph = True, create_graph=True)
loss2 = torch.mean(h1[0] - h2[0])

loss = loss1 + loss2

result = torch.autograd.grad(loss, x)
print(result)


In [17]:
import torch

def fxy(x, y):
    return x ** 2 + y ** 2

x = torch.tensor((2.0,), requires_grad=True)
y = torch.tensor((3.7,), requires_grad=True)

lr = 0.1
num_epochs = 10000

for i in range(num_epochs):
    pred = fxy(x, y)
    pred.backward()

    x.data -= lr * x.grad.data
    y.data -= lr * y.grad.data

    x.grad.zero_()
    y.grad.zero_()

    if i % 100 == 0:
        print("i = ", i, " x = ", x.item(), " y = ", y.item())

i =  0  x =  1.600000023841858  y =  2.9600000381469727
i =  100  x =  3.259258973820067e-10  y =  6.029625687631324e-10
i =  200  x =  6.639225599928508e-20  y =  1.2282564516434385e-19
i =  300  x =  1.35243404871145e-29  y =  2.502002824606578e-29
i =  400  x =  2.7549569847579833e-39  y =  5.096666848491185e-39
i =  500  x =  2.802596928649634e-45  y =  2.802596928649634e-45
i =  600  x =  2.802596928649634e-45  y =  2.802596928649634e-45
i =  700  x =  2.802596928649634e-45  y =  2.802596928649634e-45
i =  800  x =  2.802596928649634e-45  y =  2.802596928649634e-45
i =  900  x =  2.802596928649634e-45  y =  2.802596928649634e-45
i =  1000  x =  2.802596928649634e-45  y =  2.802596928649634e-45
i =  1100  x =  2.802596928649634e-45  y =  2.802596928649634e-45
i =  1200  x =  2.802596928649634e-45  y =  2.802596928649634e-45
i =  1300  x =  2.802596928649634e-45  y =  2.802596928649634e-45
i =  1400  x =  2.802596928649634e-45  y =  2.802596928649634e-45
i =  1500  x =  2.8025969286